In [ ]:
import pymysql
import pandas as pd

def getSQL(Nlimit0, Noffset0):
    sql1 = "select *\n"
    sql1 +="from fbt_input fi join cos_data cd on (fi.COS_LOT=cd.ASSY_NO and fi.COS_SERIAL=cd.COS_SERIAL)\n"
    sql1 +="join ch_power_isoa_off isoa on fi.lot=isoa.LOT and fi.serial=isoa.SERIAL\n"
    sql1 +="where cd.wafer_lot>'FT750A' and cd.wafer_lot<'FT850D'\n"
    sql1 +="limit "+str(Nlimit0)+"\n"
    sql1 +="offset "+str(Noffset0)+"\n"
    return sql1


con = pymysql.connect(host='111.222.333.444', user='username', 
                      passwd='password', db='dbname')
cur=con.cursor()

try:
   s = "DESC fbt_input"
   cur.execute(s)
   res = cur.fetchall()
   df1 = pd.DataFrame(list(res))
   s = "DESC cos_data"
   cur.execute(s)
   res = cur.fetchall()
   df2 = pd.DataFrame(list(res))
   s = "DESC ch_power_isoa_off"
   cur.execute(s)
   res = cur.fetchall()
   df3 = pd.DataFrame(list(res))
   
   df = df1.append(df2, ignore_index=True).append(df3, ignore_index=True)
   df.iloc[:,0].to_csv('data/header.csv', sep=',', index=None, header=None) 
   
   N0=0
   dN=10
   for i in range(N0, N0+dN):
        Nlimit = 10000
        Noffset = i * Nlimit
        print("saving "+str(i))
        s = getSQL(Nlimit, Noffset)
        cur.execute(s)
        res = cur.fetchall()
        dat = pd.DataFrame(list(res))
        dat.to_csv(str(i)+'.csv', sep=',', index=None, header=None) 
    
except pymysql.Error as e:
    print('pymysql.Error: ', e)
con.close()